# Agregagador de CSV's

Agrega os csv's de cada órgão em um só csv (remuneracao_servidores.csv), criando os atributos 'ORGAO', 'ANO' e 'MES'. 

### Imports

In [1]:
import pandas as pd
import os
import re

## Funções

### Lista Diretórios

In [2]:
def lista_dirs(path):
    dirs=[]
    # r = raiz, d = diretorios, f = arquivos
    for r, d, f in os.walk(path):
        for direc in d:
            if '.txt' not in direc:
                dirs.append(os.path.join(r, direc))
    return dirs

### Lista os arquivos de um diretório

In [3]:
def lista_arqs(path):
    files = []
    # r = raiz, d = diretorios, f = arquivos
    for r, d, f in os.walk(path):
        for file in f:
            if '.csv' in file:
                f_out = os.path.join(r, file)
                files.append(f_out)
    return files

### Cria o dataframe de cada órgão

In [4]:
def cria_df_orgao(path):
    files = lista_arqs(path)
    df_orgao = pd.DataFrame()
    for f in files:
        if '.csv' in f:
            df_mes = pd.read_csv(f, sep=';', header=0, decimal=',', engine='python')
            
            file_name = f.split("_")[1]
            orgao_name = path.split("/")[2]
            
            mes = file_name[4:6]
            ano = file_name[0:4]
            
            df_mes["ANO"] = ano
            df_mes["MES"] = mes
            
            df_orgao= pd.concat([df_orgao, df_mes], ignore_index=True)
            df_orgao["ORGAO"] = orgao_name
    try:
        df_orgao.drop(['Unnamed: 10'], axis=1, inplace=True)
    except:
        print('Sem Coluna Unnamed')
    return df_orgao

### Cria o dataframe geral

In [5]:
def cria_df_geral(path):
    df_geral = pd.DataFrame()
    dirs = lista_dirs(path)
    for d in dirs:
        if (str(d) == "../csv/PM-ATIVOS" or str(d) == "../csv/FUNDAÇÃO VILA OLIMPICA" or str(d) == "../csv/PRODAM"):
            continue
        print("Processando diretório:", str(d))
        nome_dir = d.split("/")[1]
        ds_orgao = pd.DataFrame()
        df_orgao = cria_df_orgao(d)
        df_geral = pd.concat([df_geral,df_orgao], ignore_index=True)
        df_geral = trata_nan(df_geral)
    return df_geral

### Trata os valores NaN

In [6]:
def trata_nan(df):
    for col in df.columns:
        if(df[col].dtype == 'object'):
            df[col] = df[col].fillna('-')
        if(df[col].dtype == 'float64'):
            df[col] = df[col].fillna(0.00)
    
    return df

### Salva o dataframe

In [11]:
def salva_df(df):
    path = r'../ds/'
    print(not os.path.exists(path))
    if (not os.path.exists(path)):
        os.mkdir(path)
    df.to_csv(path + r'remuneracao_servidores.csv')

## Main

In [12]:
path = '../csv/'
df_geral = cria_df_geral(path)
salva_df(df_geral)
df_geral

False


""
